# Load Environment Variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['HUGGINGFACE_API_KEY'] = huggingface_api_key

# Import Core Langchain Libraries

In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader # Use this
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.documents import Document

# Load and Process PDFs

In [3]:
pdf_files = [
    r"F:\DiabetIQ\LLM\PDFs\BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf",
    r"F:\DiabetIQ\LLM\PDFs\BES-Ramadan-Guideline-2020-min.pdf",
    r"F:\DiabetIQ\LLM\PDFs\Diabetes_Care_BADAS_guideline2019-3.pdf",
    r"F:\DiabetIQ\LLM\PDFs\Insulin-Guideline-min.pdf"
]

all_docs = [] # Will store LangChain Document objects

print("Loading and Processing PDFs...")
for pdf_path in pdf_files:
    try:
        # Extract filename for metadata
        file_name = os.path.basename(pdf_path)
        print(f"-> Loading: {file_name}")

        loader = PyPDFLoader(pdf_path)
        # Load pages as individual documents. Each doc will have metadata['page']
        pages = loader.load_and_split() # This does basic splitting

        # Add source filename to metadata for each page/document
        for page_doc in pages:
            page_doc.metadata['source'] = file_name
            # Optional: clean up page content slightly if needed
            # page_doc.page_content = page_doc.page_content.replace('\n', ' ').strip()

        all_docs.extend(pages)
        print(f"   Loaded {len(pages)} pages.")

    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")

print(f"\nTotal documents loaded: {len(all_docs)}")
if all_docs:
    print("\nSample Document Metadata (first doc):")
    print(all_docs[0].metadata)
    print("\nSample Document Content (first 500 chars of first doc):")
    print(all_docs[0].page_content[:500])
else:
    print("\nNo documents were loaded successfully.")
    # Consider exiting or handling this error appropriately
    exit()

Loading and Processing PDFs...
-> Loading: BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf
   Loaded 38 pages.
-> Loading: BES-Ramadan-Guideline-2020-min.pdf
   Loaded 46 pages.
-> Loading: Diabetes_Care_BADAS_guideline2019-3.pdf
   Loaded 79 pages.
-> Loading: Insulin-Guideline-min.pdf
   Loaded 93 pages.

Total documents loaded: 256

Sample Document Metadata (first doc):
{'producer': 'Nitro PDF PrimoPDF', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': '2020-06-07T20:17:39-06:00', 'moddate': '2020-06-07T20:17:39-06:00', 'title': 'Microsoft Word - BES COVID Pract Recomnd 06 June Final Copy', 'author': 'Mir', 'source': 'BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}

Sample Document Content (first 500 chars of first doc):
Bangladesh Endocrine Society (BES) 
Practical Recommendations for Management of 
Diabetes and Other Endocrine Diseases in Patients with 
COVID-19 
 
 
 
 
 
Published Online June 2020 
 
 
All rights res

# Initialize Text Splitter 

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    # Keep separators that make sense for text structure
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    length_function=len,
)

# Split Documents into Chunks

In [5]:
chunks = text_splitter.split_documents(all_docs)

print(f"\nTotal chunks created: {len(chunks)}")
if chunks:
    print("\nSample Chunk Metadata (first chunk):")
    print(chunks[0].metadata)
    print("\nSample Chunk Content (first 500 chars):")
    print(chunks[0].page_content[:500])
else:
    print("\nNo chunks were created. Check splitting process.")
    exit()


Total chunks created: 702

Sample Chunk Metadata (first chunk):
{'producer': 'Nitro PDF PrimoPDF', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': '2020-06-07T20:17:39-06:00', 'moddate': '2020-06-07T20:17:39-06:00', 'title': 'Microsoft Word - BES COVID Pract Recomnd 06 June Final Copy', 'author': 'Mir', 'source': 'BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}

Sample Chunk Content (first 500 chars):
Bangladesh Endocrine Society (BES) 
Practical Recommendations for Management of 
Diabetes and Other Endocrine Diseases in Patients with 
COVID-19 
 
 
 
 
 
Published Online June 2020 
 
 
All rights reserved by: Bangladesh Endocrine Society (BES) 
 
 
Published by 
Bangladesh Endocrine Society (BES) 
Website: http://bes-org.net 
E-mail: 
endobd2012@gmail.com


# Initialize Embeddings and Vector Store

In [6]:
print("\nInitializing Embedding Model...")
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/e5-small-v2")

print("\nCreating Vector Store (ChromaDB)...")
# Chroma.from_documents handles Document objects directly
# Consider adding persistence: persist_directory="./chroma_db_diabetiq"
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    # persist_directory="./chroma_db_diabetiq" # Uncomment to save DB locally
)
# If persisting: vectorstore.persist()

# To load later:
# vectorstore = Chroma(persist_directory="./chroma_db_diabetiq", embedding_function=embedding_model)

print("Vector Store Created.")


Initializing Embedding Model...

Creating Vector Store (ChromaDB)...
Vector Store Created.


# Create Retriever

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # Retrieve top 5 chunks

# Confirm Retriever Configuration

In [8]:
print(f"Retriever configured (using k={retriever.search_kwargs.get('k', 'default')}).")

Retriever configured (using k=5).


# Define RAG Prompt Template

In [9]:
from langchain_core.prompts import PromptTemplate
prompt_template = """
You are DiabetIQ, an AI assistant specializing in diabetes management for patients in Bangladesh, based *strictly* on the provided context documents.

Context Documents:
{context}

Based *only* on the information in the numbered context documents above, answer the following question concisely and directly.
Your advice should be actionable and consider general practices relevant to Bangladesh where possible (e.g., common foods mentioned in context, local guidelines if present in context).
Do *not* add information that is not present in the context.
If the context does not contain the answer, state that clearly.
Always conclude your response by advising the user to consult a healthcare professional for personalized medical advice.

Question: {question}

Answer:
"""

prompt = PromptTemplate.from_template(prompt_template)

# Initialize RAG LLM

In [10]:
print("Initializing LLM (Ollama - Mistral)...")
llm = OllamaLLM(model="mistral")

Initializing LLM (Ollama - Mistral)...


# Define Document Formatting Function

In [11]:
def format_docs_with_metadata(docs):
    """Formats retrieved documents including source and page."""
    formatted_strings = []
    for i, doc in enumerate(docs):
        metadata_str = f"Source: {doc.metadata.get('source', 'N/A')}, Page: {doc.metadata.get('page', 'N/A')}"
        content_str = doc.page_content.replace('\n', ' ').strip()
        formatted_strings.append(f"{i+1}. [{metadata_str}] {content_str}")
    return "\n\n".join(formatted_strings)

# Construct RAG Chain

In [12]:
rag_chain = (
    {"context": retriever | format_docs_with_metadata, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG Chain constructed.")

RAG Chain constructed.


# Re-set Environment Variables (Redundant) 

In [13]:

import os
from dotenv import load_dotenv

load_dotenv()

langchain_api_key = os.getenv('LANGCHAIN_API_KEY', 'YOUR_LANGCHAIN_API_KEY_FALLBACK') # Add fallbacks or ensure .env is present
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY', 'YOUR_HF_API_KEY_FALLBACK') # Add fallbacks or ensure .env is present
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['HUGGINGFACE_API_KEY'] = huggingface_api_key

print("Environment variables set.")

Environment variables set.


# Import DeepEval and Related Libraries

In [23]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate


import deepeval
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.models import OllamaModel

print("Libraries imported.")

Libraries imported.


# RAG Pipeline

In [17]:
pdf_files = [
    r"F:\\DiabetIQ\\LLM\\PDFs\\BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf",
    r"F:\\DiabetIQ\\LLM\\PDFs\\BES-Ramadan-Guideline-2020-min.pdf",
    r"F:\\DiabetIQ\\LLM\\PDFs\\Diabetes_Care_BADAS_guideline2019-3.pdf",
    r"F:\\DiabetIQ\\LLM\\PDFs\\Insulin-Guideline-min.pdf"
]


all_docs = []
print("Loading and Processing PDFs for RAG chain...")
for pdf_path in pdf_files:
    try:
        file_name = os.path.basename(pdf_path)
        print(f"-> Loading: {file_name}")
        loader = PyPDFLoader(pdf_path)
        pages = loader.load() 
        for page_doc in pages:
            page_doc.metadata['source'] = file_name
           
        all_docs.extend(pages)
        print(f"   Loaded {len(pages)} pages.")
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")
print(f"Total documents loaded: {len(all_docs)}")
if not all_docs:
    print("No documents loaded, exiting.")
    exit()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    length_function=len,
)
chunks = text_splitter.split_documents(all_docs)
print(f"Total chunks created: {len(chunks)}")
if not chunks:
    print("No chunks created, exiting.")
    exit()


print("Initializing Embedding Model...")
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/e5-small-v2")
print("Creating Vector Store (ChromaDB)...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
)
print("Vector Store Created.")


retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
print(f"Retriever configured (using k={retriever.search_kwargs.get('k', 'default')}).")


prompt_template = """
You are DiabetIQ, an AI assistant specializing in diabetes management for patients in Bangladesh, based *strictly* on the provided context documents.

Context Documents:
{context}

Based *only* on the information in the numbered context documents above, answer the following question concisely and directly.
Your advice should be actionable and consider general practices relevant to Bangladesh where possible (e.g., common foods mentioned in context, local guidelines if present in context).
Do *not* add information that is not present in the context.
If the context does not contain the answer, state that clearly.
Always conclude your response by advising the user to consult a healthcare professional for personalized medical advice.

Question: {question}

Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)


print("Initializing Generation LLM (Ollama - Mistral)...")
llm = OllamaLLM(model="mistral") 


def format_docs_with_metadata(docs):
    formatted_strings = []
    for i, doc in enumerate(docs):
        metadata_str = f"Source: {doc.metadata.get('source', 'N/A')}, Page: {doc.metadata.get('page', 'N/A')}"
        content_str = doc.page_content.replace('\n', ' ').strip()
        formatted_strings.append(f"{i+1}. [{metadata_str}] {content_str}")
    return "\n\n".join(formatted_strings)

rag_chain = (
    {"context": retriever | format_docs_with_metadata, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("RAG Chain constructed and ready for testing.")

def retrieve_context(question: str):
    """Retrieves raw document chunks for a given question."""
    return retriever.invoke(question)

print("Context retrieval function defined.")

Loading and Processing PDFs for RAG chain...
-> Loading: BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf
   Loaded 38 pages.
-> Loading: BES-Ramadan-Guideline-2020-min.pdf
   Loaded 48 pages.
-> Loading: Diabetes_Care_BADAS_guideline2019-3.pdf
   Loaded 79 pages.
-> Loading: Insulin-Guideline-min.pdf
   Loaded 48 pages.
Total documents loaded: 213
Total chunks created: 684
Initializing Embedding Model...
Creating Vector Store (ChromaDB)...
Vector Store Created.
Retriever configured (using k=5).
Initializing Generation LLM (Ollama - Mistral)...
RAG Chain constructed and ready for testing.
Context retrieval function defined.


# Configure DeepEval Evaluation Model

In [24]:
import os 

print("Configuring DeepEval Evaluation Model (using deepeval.models.OllamaModel)...")


ollama_model_name = "llama3.2"

ollama_base_url = os.getenv("OLLAMA_BASE_URL", None) 

if ollama_base_url:
    print(f"Using Ollama base URL: {ollama_base_url}")
    evaluation_model = OllamaModel(model=ollama_model_name, base_url=ollama_base_url)
else:
    print("Using default Ollama base URL (http://localhost:11434).")
    
    evaluation_model = OllamaModel(model=ollama_model_name)

print(f"DeepEval Evaluation Model ({ollama_model_name}) configured.")

Configuring DeepEval Evaluation Model (using deepeval.models.OllamaModel)...
Using default Ollama base URL (http://localhost:11434).
DeepEval Evaluation Model (llama3.2) configured.


In [25]:
print("Defining DeepEval Metrics...")


print(f"Using configured '{evaluation_model.model_name}' DeepEval model object for judgments.")


answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.7,
    model=evaluation_model,          
    include_reason=True
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model=evaluation_model,           
    include_reason=True
)

contextual_relevancy_metric = ContextualRelevancyMetric(
    threshold=0.7,
    model=evaluation_model,           
    include_reason=True
)


metrics = [
    answer_relevancy_metric,
    faithfulness_metric,
    contextual_relevancy_metric
]
print("DeepEval Metrics defined.")

Defining DeepEval Metrics...
Using configured 'llama3.2' DeepEval model object for judgments.
DeepEval Metrics defined.


In [26]:
print("Generating Test Cases and RAG Outputs...")


test_questions = [
    "How can I control my blood sugar level with diet according to the textbook?",
    "What does the BADAS guideline say about insulin initiation?",
    "Tell me about managing diabetes during Ramadan based on the provided texts.",
    "I have diabetes. Can I eat sweets?",
    "What are the symptoms of hypoglycemia?", 
    "Does the context mention specific insulin brand names?", 
    "Summarize the COVID-19 recommendations for diabetic patients."
]

test_cases = []

for question in test_questions:
    print(f"\nProcessing question: {question}")
    try:
        
        retrieved_docs = retrieve_context(question)
        retrieval_context_list = [doc.page_content for doc in retrieved_docs] 

        
        actual_output = rag_chain.invoke(question)
        print(f"  - Generated Output: {actual_output[:150]}...") 

        
        test_case = LLMTestCase(
            input=question,
            actual_output=actual_output,
            retrieval_context=retrieval_context_list 
        )
        test_cases.append(test_case)
        print("  - Test case created.")

    except Exception as e:
        print(f"  - Error processing question '{question}': {e}")

print(f"\nTotal test cases generated: {len(test_cases)}")

Generating Test Cases and RAG Outputs...

Processing question: How can I control my blood sugar level with diet according to the textbook?
  - Generated Output:  To help control your blood sugar level according to the textbooks, consider following a balanced diet that emphasizes fruits, legumes, whole grains, ...
  - Test case created.

Processing question: What does the BADAS guideline say about insulin initiation?
  - Generated Output:  According to the BADAS Guideline 2019, in all major surgeries, glucose-insulin infusion should be started. However, it does not specify the exact typ...
  - Test case created.

Processing question: Tell me about managing diabetes during Ramadan based on the provided texts.
  - Generated Output:  To manage diabetes during Ramadan, it's crucial to adjust medication doses, diet, and exercise regimens as recommended. The following are some genera...
  - Test case created.

Processing question: I have diabetes. Can I eat sweets?
  - Generated Output:  Whil

In [28]:
print("\n--- Starting DeepEval Evaluation (Synchronous Mode) ---") 

if not test_cases:
    print("No test cases were generated. Cannot run evaluation.")
else:
    
    evaluation_results = evaluate(
        test_cases=test_cases,
        metrics=metrics,
        run_async=False  
    )

    print("\n--- DeepEval Evaluation Complete ---")

    
    print("\nEvaluation Results Summary:")
    
    pass

print("\nEvaluation finished.")


--- Starting DeepEval Evaluation (Synchronous Mode) ---


✨ You're running DeepEval's latest Answer Relevancy Metric! (using llama3.2 (Ollama), strict=False, 
async_mode=False)...

✨ You're running DeepEval's latest Faithfulness Metric! (using llama3.2 (Ollama), strict=False, 
async_mode=False)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using llama3.2 (Ollama), strict=False, 
async_mode=False)...

Evaluating 7 test case(s) sequentially: |█████████████████████████████████████████████████████|100% (7/7) [Time Taken: 03:46, 32.42s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.7777777777777778, threshold: 0.7, strict: False, evaluation model: llama3.2 (Ollama), reason: The score is 0.78 because while the output contains some relevant information about diet and glycemic control, there are two irrelevant statements that detract from its usefulness: one general statement on overall health and another that focuses on improving weight management rather than specifically addressing blood sugar levels with diet., error: None)
  - ✅ Faithfulness (score: 0.7333333333333333, threshold: 0.7, strict: False, evaluation model: llama3.2 (Ollama), reason: The score is 0.73 because the actual output contains contradictions regarding Einstein's nationality, the year he won the Nobel Prize, and the effectiveness of dietary supplementation for people with diabetes., error: None)
  - ✅ Contextual Relevancy (score: 0.7666666666666667, threshold: 0.7, strict: False, evaluation model: llama3.2 (Ollama), reason: The score is 0.77 b

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


--- DeepEval Evaluation Complete ---

Evaluation Results Summary:

Evaluation finished.
